In [1]:
!git clone https://github.com/kimbold/VRDL_2019

Cloning into 'VRDL_2019'...
remote: Enumerating objects: 250, done.
remote: Counting objects: 100% (250/250), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 4115 (delta 85), reused 230 (delta 71), pack-reused 3865
Receiving objects: 100% (4115/4115), 55.42 MiB | 48.67 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [2]:
%cd VRDL_2019/HW3

/content/VRDL_2019/HW3


**Load svhn dataset**

In [3]:
!wget http://ufldl.stanford.edu/housenumbers/train.tar.gz
!wget http://ufldl.stanford.edu/housenumbers/test.tar.gz

--2019-11-25 07:52:05--  http://ufldl.stanford.edu/housenumbers/train.tar.gz
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 404141560 (385M) [application/x-gzip]
Saving to: ‘train.tar.gz’

train.tar.gz        100%[===================>] 385.42M  63.6MB/s    in 6.9s    

2019-11-25 07:52:12 (56.2 MB/s) - ‘train.tar.gz’ saved [404141560/404141560]

--2019-11-25 07:52:15--  http://ufldl.stanford.edu/housenumbers/test.tar.gz
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276555967 (264M) [application/x-gzip]
Saving to: ‘test.tar.gz’

test.tar.gz         100%[===================>] 263.74M  65.5MB/s    in 4.9s    

2019-11-25 07:52:20 (53.7 MB/s) - ‘test.tar.gz’ saved [276555967/2

**Extract dataset files**

In [4]:
!tar -zxvf train.tar.gz
!tar -zxvf test.tar.gz

train/
train/4804.png
train/4777.png
train/4759.png
train/4750.png
train/4727.png
train/4722.png
train/4698.png
train/4685.png
train/4655.png
train/4621.png
train/4527.png
train/4496.png
train/4492.png
train/4437.png
train/4394.png
train/4388.png
train/4380.png
train/4377.png
train/4325.png
train/4312.png
train/4298.png
train/4182.png
train/4145.png
train/4140.png
train/4090.png
train/4078.png
train/4034.png
train/4009.png
train/3996.png
train/3957.png
train/3934.png
train/3919.png
train/3896.png
train/3858.png
train/3857.png
train/3767.png
train/3697.png
train/3689.png
train/3668.png
train/3664.png
train/3618.png
train/3553.png
train/3535.png
train/3462.png
train/3415.png
train/3401.png
train/3367.png
train/3362.png
train/3359.png
train/3162.png
train/3161.png
train/3090.png
train/3057.png
train/2974.png
train/2967.png
train/2959.png
train/2949.png
train/2932.png
train/2925.png
train/2916.png
train/2905.png
train/2882.png
train/2738.png
train/2689.png
train/2613.png
train/2564.png
tra

**Import required modules**

In [5]:
import numpy as np
import argparse
import os
import json
from yolo.frontend import create_yolo, get_object_labels
import xml.etree.cElementTree as ET
import h5py
from lxml import etree

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


**Define methods to load annotations**

In [0]:
def get_name(index, hdf5_data):
    name = hdf5_data['/digitStruct/name']
    return ''.join([chr(v[0]) for v in hdf5_data[name[index][0]].value])

def keys(f):
    return [key for key in f.keys()]

def get_bbox(index, hdf5_data):
  """
  The box data contains width and height of the box as well as the upper left corner.
  By supplying the distance from the left border and top border, the upper left point can be identified.
  """
  attrs = {}
  item = hdf5_data['digitStruct']['bbox'][index].item()
  for key in ['label', 'left', 'top', 'width', 'height']:
      attr = hdf5_data[item][key]
      values = [hdf5_data[attr.value[i].item()].value[0][0]
                for i in range(len(attr))] if len(attr) > 1 else [attr.value[0][0]]
      attrs[key] = values
  return attrs

def prettyPrintXml(xmlFilePathToPrettyPrint):
    parser = etree.XMLParser(resolve_entities=False, strip_cdata=False)
    document = etree.parse(xmlFilePathToPrettyPrint, parser)
    document.write(xmlFilePathToPrettyPrint, pretty_print=True, encoding='utf-8')

# replacement strings
WINDOWS_LINE_ENDING = b'\r\n'
UNIX_LINE_ENDING = b'\n'

def replace_Unix_with_Windows_in_XLM(xmlFilePath):
    with open(xmlFilePath, 'rb') as open_file:
      content = open_file.read()

    content = content.replace(UNIX_LINE_ENDING,WINDOWS_LINE_ENDING)

    with open(xmlFilePath, 'wb') as open_file:
        open_file.write(content)

In [0]:
import os
directory = "tests/dataset/svhn/test_anns"
if not os.path.exists(directory):
    os.makedirs(directory)

**Load annotations for training**

In [0]:
# Need to define method to read .mat data and then write annotation files for each picture
filename = "train/digitStruct.mat"

f = h5py.File(filename,'r')

# Iterate through all images to create the annotation file
for image in range(f['digitStruct']['bbox'].shape[0]):
#for image in range(3):
  #annotation
  root = ET.Element("annotation")

  #Get image annotation data
  size_data = get_bbox(image, f)

  #filename
  filename = get_name(image,f)
  doc = ET.SubElement(root, "filename").text = filename

  # For each detected digit, add the data for the box for it
  object_list = [[] for _ in range(len(size_data['label']))]
  bndbox = [[] for _ in range(len(size_data['label']))]

  for number in range(len(size_data['label'])):
    object_list[number] = ET.SubElement(root, "object")
    ET.SubElement(object_list[number], "name").text = str(int(size_data['label'][number]))

    bndbox[number] = ET.SubElement(object_list[number], "bndbox")
    ET.SubElement(bndbox[number], "xmin").text = str(int(size_data['left'][number]))
    ET.SubElement(bndbox[number], "ymin").text = str(int(size_data['top'][number]))
    ET.SubElement(bndbox[number], "xmax").text = str(int(size_data['width'][number]+size_data['left'][number]))
    ET.SubElement(bndbox[number], "ymax").text = str(int(size_data['height'][number]+size_data['top'][number]))
  
  tree = ET.ElementTree(root)
  tree.write("tests/dataset/svhn/anns/"+str(image+1)+".xml")
  prettyPrintXml("tests/dataset/svhn/anns/"+str(image+1)+".xml")
  replace_Unix_with_Windows_in_XLM("tests/dataset/svhn/anns/"+str(image+1)+".xml")

if os.path.exists('tests/dataset/svhn/anns/.ipynb_checkpoints'):
  os.rmdir('tests/dataset/svhn/anns/.ipynb_checkpoints')

In [0]:
# Need to define method to read .mat data and then write annotation files for each picture
enabled=0
if(enabled==1):
  filename = "train/digitStruct.mat"

  f = h5py.File(filename,'r')

  # Iterate through all images to create the annotation file
  #for image in range(len(f['digitStruct']['bbox'].shape[0])):
  for image in [2,3]:
    #annotation
    root = ET.Element("annotation")

    #Get image annotation data
    size_data = get_bbox(image, f)

    #filename
    filename = get_name(image,f)
    doc = ET.SubElement(root, "filename").text = filename

    # For each detected digit, add the data for the box for it
    object_list = [[] for _ in range(len(size_data['label']))]
    bndbox = [[] for _ in range(len(size_data['label']))]

    for number in range(len(size_data['label'])):
      object_list[number] = ET.SubElement(root, "object")
      ET.SubElement(object_list[number], "name").text = str(int(size_data['label'][number]))

      bndbox[number] = ET.SubElement(object_list[number], "bndbox")
      ET.SubElement(bndbox[number], "xmin").text = str(int(size_data['left'][number]))
      ET.SubElement(bndbox[number], "ymin").text = str(int(size_data['top'][number]))
      ET.SubElement(bndbox[number], "xmax").text = str(int(size_data['width'][number]+size_data['left'][number]))
      ET.SubElement(bndbox[number], "ymax").text = str(int(size_data['height'][number]+size_data['top'][number]))

    
    tree = ET.ElementTree(root)
    tree.write("tests/dataset/svhn/test_anns/"+str(image+1)+".xml")
    prettyPrintXml("tests/dataset/svhn/test_anns/"+str(image+1)+".xml")
    replace_Unix_with_Windows_in_XLM("tests/dataset/svhn/test_anns/"+str(image+1)+".xml")
    
  if os.path.exists('tests/dataset/svhn/test_anns/.ipynb_checkpoints'):
    os.rmdir('tests/dataset/svhn/test_anns/.ipynb_checkpoints')

In [10]:
!dir tests/dataset/svhn/anns
!dir tests/dataset/svhn/test_anns

10000.xml  14297.xml  18592.xml  22887.xml  27181.xml  31476.xml  5708.xml
10001.xml  14298.xml  18593.xml  22888.xml  27182.xml  31477.xml  5709.xml
10002.xml  14299.xml  18594.xml  22889.xml  27183.xml  31478.xml  570.xml
10003.xml  1429.xml   18595.xml  2288.xml   27184.xml  31479.xml  5710.xml
10004.xml  142.xml    18596.xml  22890.xml  27185.xml  3147.xml   5711.xml
10005.xml  14300.xml  18597.xml  22891.xml  27186.xml  31480.xml  5712.xml
10006.xml  14301.xml  18598.xml  22892.xml  27187.xml  31481.xml  5713.xml
10007.xml  14302.xml  18599.xml  22893.xml  27188.xml  31482.xml  5714.xml
10008.xml  14303.xml  1859.xml	 22894.xml  27189.xml  31483.xml  5715.xml
10009.xml  14304.xml  185.xml	 22895.xml  2718.xml   31484.xml  5716.xml
1000.xml   14305.xml  18600.xml  22896.xml  27190.xml  31485.xml  5717.xml
10010.xml  14306.xml  18601.xml  22897.xml  27191.xml  31486.xml  5718.xml
10011.xml  14307.xml  18602.xml  22898.xml  27192.xml  31487.xml  5719.xml
10012.xml  14308.xml  18603.x

**Train yolo**

Training yolo takes around 3 hours for all layers here
and then 2 more hours for the final layer.

In [0]:
#Train and validate YOLO_v2 model on any dataset
def setup_training(config_file):
    """make directory to save weights & its configuration """
    import shutil
    with open(config_file) as config_buffer:
        config = json.loads(config_buffer.read())
    dirname = config['train']['saved_folder']
    if os.path.isdir(dirname):
        print("{} is already exists. Weight file in directory will be overwritten".format(dirname))
    else:
        print("{} is created.".format(dirname, dirname))
        os.makedirs(dirname)
    print("Weight file and Config file will be saved in \"{}\"".format(dirname))
    shutil.copyfile(config_file, os.path.join(dirname, "config.json"))
    return config, os.path.join(dirname, "weights.h5")


def train(conf="configs/from_scratch.json"):
  #path to configuration file

    config, weight_file = setup_training(conf)
    
    if config['train']['is_only_detect']:
        labels = ["object"]
    else:
        if config['model']['labels']:
            labels = config['model']['labels']
        else:
            labels = get_object_labels(config['train']['train_annot_folder'])
    print(labels)

    # 1. Construct the model 
    yolo = create_yolo(config['model']['architecture'],
                       labels,
                       config['model']['input_size'],
                       config['model']['anchors'],
                       config['model']['coord_scale'],
                       config['model']['class_scale'],
                       config['model']['object_scale'],
                       config['model']['no_object_scale'])

    # 2. Load the pretrained weights (if any) 
    yolo.load_weights(config['pretrained']['full'], by_name=True)

    # 3. actual training 
    yolo.train(config['train']['train_image_folder'],
               config['train']['train_annot_folder'],
               config['train']['actual_epoch'],
               weight_file,
               config["train"]["batch_size"],
               config["train"]["jitter"],
               config['train']['learning_rate'], 
               config['train']['train_times'],
               config['train']['valid_times'],
               config['train']['valid_image_folder'],
               config['train']['valid_annot_folder'],
               config['train']['first_trainable_layer'],
               config['train']['is_only_detect'])
    # loss: 2.1691, train batch jitter=False


In [0]:
train(conf="configs/from_scratch_custom.json")

svhn is created.
Weight file and Config file will be saved in "svhn"
['object']














/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 422, 422, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
___________________________

In [0]:
train(conf="configs/from_scratch2_custom.json")

**After training the weights are stored in .h5 file**

So now it is necessary to ensure that it is closed properly to avoid corruption.

In [0]:
# Closing all .hd5 files: https://stackoverflow.com/questions/29863342/close-an-open-h5py-data-file against corruption
import tables
tables.file._open_files.close_all()

**Evaluate trained yolo digit detector**

In [0]:
import argparse
import json
import cv2
import numpy as np
from yolo.frontend import create_yolo
from yolo.backend.utils.box import draw_scaled_boxes
from yolo.backend.utils.annotation import parse_annotation
from yolo.backend.utils.eval._box_match import BoxMatcher

import os
import yolo

DEFAULT_CONFIG_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "config.json")
DEFAULT_WEIGHT_FILE = os.path.join(yolo.PROJECT_ROOT, "svhn", "weights.h5")

DEFAULT_THRESHOLD = 0.3


In [0]:
def count_true_positives(detect_boxes, true_boxes, detect_labels=None, true_labels=None):
    """
    # Args
        detect_boxes : array, shape of (n_detected_boxes, 4)
        true_boxes : array, shape of (n_true_boxes, 4)
        detected_labels : array, shape of (n_detected_boxes,)
        true_labels :
    """
    n_true_positives = 0
    print("Detect boxes:",detect_boxes,"True boxes", true_boxes)
    matcher = BoxMatcher(detect_boxes, true_boxes, detect_labels, true_labels)
    for i in range(len(detect_boxes)):
        matching_idx, iou = matcher.match_idx_of_box1_idx(i)
        # print("detect_idx: {}, true_idx: {}, matching-score: {}".format(i, matching_idx, iou))
        if matching_idx is not None and iou > 0.5:
            n_true_positives += 1
    return n_true_positives


def calc_score(n_true_positives, n_truth, n_pred):
    """
    # Args
        detect_boxes : list of box-arrays
        true_boxes : list of box-arrays
    """
    print("True positives:", n_true_positives)
    print("n_pred:", n_pred)
    print("n_truth:", n_truth)
    precision = n_true_positives / n_pred
    recall = n_true_positives / n_truth
    print("Precision:", precision)
    print("Recall:", recall)
    fscore = 2* precision * recall / (precision + recall)
    score = {"fscore": fscore, "precision": precision, "recall": recall}
    return score

In [0]:
def evaluate(conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE, threshold=DEFAULT_THRESHOLD):

      with open(conf) as config_buffer:
          config = json.loads(config_buffer.read())

      # 2. create yolo instance & predict
      yolo = create_yolo(config['model']['architecture'],
                        config['model']['labels'],
                        config['model']['input_size'],
                        config['model']['anchors'])
      yolo.load_weights(weights)

      # 3. read image
      write_dname = "detected"
      if not os.path.exists(write_dname): os.makedirs(write_dname)
      annotations = parse_annotation(config['train']['valid_annot_folder'],
                                    config['train']['valid_image_folder'],
                                    config['model']['labels'],
                                    is_only_detect=config['train']['is_only_detect'])

      n_true_positives = 0
      n_truth = 0
      n_pred = 0
      for i in range(len(annotations)):
          img_path = annotations.fname(i)
          img_fname = os.path.basename(img_path)
          image = cv2.imread(img_path)
          true_boxes = annotations.boxes(i)
          true_labels = annotations.code_labels(i)
          
          boxes, probs = yolo.predict(image, float(threshold))
          labels = np.argmax(probs, axis=1) if len(probs) > 0 else [] 
        
          # 4. save detection result
          image = draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
          output_path = os.path.join(write_dname, os.path.split(img_fname)[-1])
          
          cv2.imwrite(output_path, image)
          print("{}-boxes are detected. {} saved.".format(len(boxes), output_path))

          n_true_positives += count_true_positives(boxes, true_boxes, labels, true_labels)
          n_truth += len(true_boxes)
          n_pred += len(boxes)
      print(calc_score(n_true_positives, n_truth, n_pred))

In [18]:
#evaluate()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 422, 422, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [20]:
#evaluate(weights="/content/VRDL_2019/HW3/svhn/weights.h5", threshold=0.3)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 416, 416, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 422, 422, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 208, 208, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 208, 208, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)



Detect boxes: [[16  5 24 19]
 [24  5 33 19]] True boxes [[17  5 25 20]
 [25  5 34 20]]
True positives: 6
n_pred: 6
n_truth: 6
Precision: 1.0
Recall: 1.0
{'fscore': 1.0, 'precision': 1.0, 'recall': 1.0}


/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:127: DeprecationWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


**Predict test data**

In [1]:
import json


def create_yolo_instance(conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE):
  with open(conf) as config_buffer:
    config = json.loads(config_buffer.read())
  
  # 2. create yolo instance & predict
  yolo = create_yolo(config['model']['architecture'],
                     config['model']['labels'],
                     config['model']['input_size'],
                     config['model']['anchors'])
  yolo.load_weights(weights)
  return yolo


def predict_boxes_digits(image_number=10, image_folder="test/",conf=DEFAULT_CONFIG_FILE, weights=DEFAULT_WEIGHT_FILE, threshold=DEFAULT_THRESHOLD):

      # Create list of dictionaries for submission
      prediction_dictionaries = [{} for _ in range(image_number)]


      with open(conf) as config_buffer:
          config = json.loads(config_buffer.read())

      # 2. create yolo instance & predict
      yolo = create_yolo(config['model']['architecture'],
                        config['model']['labels'],
                        config['model']['input_size'],
                        config['model']['anchors'])
      yolo.load_weights(weights)

      # 3. read image
      write_dname = "detected"
      if not os.path.exists(write_dname): os.makedirs(write_dname)
  
      for i in range(1,image_number):
          # For each image, get the predicted labels, probabilities and boxes
          bbox = []
          score = []
          label = []

          img_path=image_folder+str(i)+".png"
          print(img_path)
          img_fname = os.path.basename(img_path)
          image = cv2.imread(img_path)
          
          boxes, probs = yolo.predict(image, float(threshold))
          labels = np.argmax(probs, axis=1) if len(probs) > 0 else [] 
          for label in range(len(labels)):
            prediction_dictionaries[i-1]['bbox']=boxes.tolist()[label]
            prediction_dictionaries[i-1]['score']=np.max(probs.tolist()[label])
            prediction_dictionaries[i-1]['label']=labels.tolist()[label]

          
          # 4. save detection result
          image = draw_scaled_boxes(image, boxes, probs, config['model']['labels'])
          output_path = os.path.join(write_dname, os.path.split(img_fname)[-1])
          
          cv2.imwrite(output_path, image)
          print("{}-boxes are detected. {} saved.".format(len(boxes), output_path))
      
      with open('0845058.json', 'w') as f:
          f.write(json.dumps(prediction_dictionaries))



NameError: ignored

**Timing for one picture**

In [0]:
#Create yolo instance
#weights="/content/VRDL_2019/HW3/svhn/weights.h5"
conf = os.path.join(yolo.PROJECT_ROOT, "svhn", "config_customized.json")
yolo = create_yolo_instance(conf=conf, weights=DEFAULT_WEIGHT_FILE)

#Load image from test folder (unseen)
image_folder="test/"
i=1 # load the first image
img_path=image_folder+str(i)+".png"
img_fname = os.path.basename(img_path)
image = cv2.imread(img_path)

In [0]:
%%timeit
yolo.predict(image, float(0.3))

In [0]:
weights="/content/VRDL_2019/HW3/svhn/weights.h5"
weights_1="/content/VRDL_2019/HW3/weights.h5"

#Load config, define folder, load weights and then go through all images 
predict_boxes_digits(image_number=13068,conf="/content/VRDL_2019/HW3/testing.json",image_folder="test/", weights=weights)